# transfer Learning

## Import libraries

In [1]:
import os
import json
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms.v2 as v2

from torchvision import models
from tqdm import tqdm

## Folder

In [ ]:
!unzip /content/dataset_dogs.zip

In [3]:
import shutil
shutil.move('/content/dogs/test/format.json', '/content/dogs')
os.remove('/content/dogs/train/format.json')

## Class Dataset

In [4]:
class DogDataset(data.Dataset):
  def __init__(self, path, train=True, transform = None):
    self.path = os.path.join(path, 'train' if train else 'test')
    self.transform = transform

    with open(os.path.join(path, 'format.json'), 'r') as f:
      self.format = json.load(f)

    self.length = 0
    self.files = []
    self.targets = torch.eye(10)

    for _dir, _target in self.format.items():
      path = os.path.join(self.path, _dir)
      list_file = os.listdir(path)
      self.length += len(list_file)
      self.files.extend(map(lambda _x: (os.path.join(path, _x), _target), list_file))


  def __getitem__(self, item):
    path_file, target = self.files[item]
    t = self.targets[target]
    img = Image.open(path_file)

    if self.transform:
      img = self.transform(img)

    return img, t

  def __len__(self):
    return self.length

## Model

In [5]:
resnet_w = models.ResNet50_Weights.DEFAULT
transforms = resnet_w.transforms()

model = models.resnet50(weights=resnet_w)
model.requires_grad_(False)
model.fc = nn.Linear(512*4, 10)
model.fc.requires_grad_(True)

Linear(in_features=2048, out_features=10, bias=True)

## Data Loader

In [6]:
batch=32

d_train = DogDataset('dogs', transform=transforms)
train_loader = data.DataLoader(d_train, batch_size=batch, shuffle=True, num_workers=4)

d_test = DogDataset('dogs', train=False, transform=transforms)
test_loader = data.DataLoader(d_test, batch_size=batch, shuffle=True, num_workers=4)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Train

In [7]:
optimizer = optim.Adam(params=model.fc.parameters(), lr=0.001, weight_decay=0.001)
loss_f = nn.CrossEntropyLoss()
epochs=3

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
model.to(device)
model.train()
for _e in range(epochs):
  loss_mean = 0
  lm_c = 0

  train_tqdm = tqdm(train_loader, leave=True)
  for x_train, y_train in train_tqdm:
    x_train, y_train = x_train.to(device), y_train.to(device)

    predict = model(x_train)
    loss = loss_f(predict, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    lm_c += 1
    loss_mean = 1/lm_c*loss.item() + (1-1/lm_c)*loss_mean
    train_tqdm.set_description(f'train [epoch:{_e+1 }] loss_mean={loss_mean:.3f}')

train [epoch:3] loss_mean=0.329: 100%|██████████| 51/51 [00:07<00:00,  6.47it/s]


## Test

In [10]:
Q = 0
P = 0
count = 0

model.eval()
test_tqdm = tqdm(test_loader, leave=True)
for x_test, y_test in test_tqdm:
  x_test, y_test = x_test.to(device), y_test.to(device)
  with torch.no_grad():
    pred = model(x_test)
    Q += loss_f(pred, y_test)

    pred = torch.argmax(pred, dim=1)
    y = torch.argmax(y_test, dim=1)
    P += torch.sum(pred == y).item()

    count += 1


print(f'\n{(Q/count).item():.3f}, {P/len(d_test):.3f}')

100%|██████████| 10/10 [00:02<00:00,  3.69it/s]


0.328, 0.940
